# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)
print(data.race.value_counts())
print(data.call.value_counts())

b    2435
w    2435
Name: race, dtype: int64
0.0    4478
1.0     392
Name: call, dtype: int64


In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
    <p>1. Since we are looking at the proportion of callback rates between applicants with black-sounding names and those with white-sounding names, a two-sample z-test for the difference of proportions would be appropriate here. The central limit theorem would apply, since the sample was random, the sample size is large enough at over 4000 for us to assume normality, and that sample size is also definitely less than 10% of the total job-seeking population, making it an independent sample. </p>
    <p>2. Our null hypothesis is that the difference between the population proportion of black people who received callbacks is equal to the population proportion of white people who received callbacks. </p>
</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

For our hypothesis test, we first use the frequentist statistical approach to find the margin of error, confidence interval, and p-value. 

In [62]:
w_total=len(w)
b_total=len(b)
w_prop=sum(data[data.race=='w'].call)/w_total #proportion of white applicants with callbacks
b_prop=sum(data[data.race=='b'].call)/b_total #proportion of black candidates with callbacks
w_var=(w_prop*(1-w_prop))/w_total
b_var=(b_prop*(1-b_prop))/b_total
print("w_var: "+str(w_var))
w_var2=data[data['race']=='w']['call'].var()
print(w_var2)
mean_proportion_diff=w_prop-b_prop
std_proportion_diff=np.sqrt(w_var+b_var)
print(std_proportion_diff)
# Our margin of error is for a 95% confidence interval, so we need to find the z-value that we multiply by the 
# standard deviation of the proportion difference. 
z_star=stats.norm.ppf(.975)
MoE=z_star*std_proportion_diff
print("margin of error "+str(np.round(MoE,8)))
conf_interval=np.array([std_proportion_diff-MoE,std_proportion_diff+MoE])
print("95% confidence interval: "+str(conf_interval))

# Assume null is true - assume that w_prop = b_prop, 
p=sum(data.call)/len(data)
std_samp_dist=np.sqrt(p*(1-p)/len(w) + p*(1-p)/len(b))
z_0=mean_proportion_diff/std_samp_dist
p_val=stats.norm.pdf(z_0)*2
print("p value: "+str(np.round(p_val,8)))
# Your solution to Q3 here

w_var: 3.580911983304638e-05
0.08723103
0.0077833705866767544
margin of error 0.01525513
95% confidence interval: [-0.00747176  0.0230385 ]
p value: 0.00017246


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

As our data is categorical and we are looking at whether one categorical variable (whether the applicant got a callback for their resume) is dependent on another (the applicant's race), a chi-square association test would be appropriate for this problem. Because our sample was random (in that resumes were randomly assigned black-sounding or white-sounding names) and the sample size is large while still being less than 10% of the population of people applying for jobs in the US, we can assume the CLT applies here. 